In [ ]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests

import requests
from bs4 import BeautifulSoup
import time
import re
import json
import string
import pandas as pd
import csv
import math
import os
import tweepy as tw
import pandas as pd
import string
import random
import datetime
import threading
import json
import math
import glob
import csv
import zipfile
import zlib
from tweepy import TweepError
from time import sleep

!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
consumer_key= 'PFntvVTEGYxOj0DpsQGLfcHih'
consumer_secret= 'qaMWaTihgx7WKCxs2CWf6VkJrGQMaQyBkd22KW6vhdHpRmO7yg'
access_token= '994786472900333570-6qqX2SH8XbHXAbJRkWdwzge4A7xktPo'
access_token_secret= 'ZryyU4n78wJyUQaNVDPNR7X7a7XKDAOtxy768SCMH2Y2E'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
class TwitterInformation:

    def __init__(self, userID, sleepTime, verbose):
        self.userID = userID
        self.tweets = []
        self.oldest_id = 0
        self.sleepTime = sleepTime
        self.verbose = verbose

    def initTweet(self, userID):
        self.tweets = api.user_timeline(screen_name = userID,
                           # 200 is the maximum allowed count
                           count = 5,
                           include_rts = True,
                           # Necessary to keep full_text
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
        print(f'now extracting {userID}')
    def allTweets(self, userID):
        try:
            self.oldest_id = self.tweets[-1].id
        except:
            print("No Tweets from this account %s" % userID)

        while True:
            try:
                extract_tweets = api.user_timeline(screen_name=userID,
                                    # 200 is the maximum allowed count
                                    count=200,
                                    include_rts = True,
                                    max_id = self.oldest_id - 1,
                                    # Necessary to keep full_text
                                    # otherwise only the first 140 words are extracted
                                    tweet_mode = 'extended'
                                    )

                if len(extract_tweets) == 0:
                    break

                self.oldest_id = extract_tweets[-1].id
                self.tweets.extend(extract_tweets)
                print('{} tweets has been downloaded'.format(len(self.tweets)))
            except tw.TweepError:
                print('Protected Tweets')


    def write_file(self, tweets):
        outtweets = [[tweet.id_str,
                      tweet.created_at,
                      str(f'https:/twitter.com/{userID}/status/{tweet.id_str}'),
                      tweet.favorite_count,
                      tweet.retweet_count,
                      tweet.full_text.encode("utf-8").decode("utf-8")]
                     for idx,tweet in enumerate(self.tweets)]
        dt = {'id':str, "created_at": str, "link": str, "favorite_count": str, "retweet_count":str, "text":str }
        df = pd.DataFrame(outtweets,columns=["id","created_at","link","favorite_count","retweet_count", "text"])
        df.astype(dt)
        df.to_csv(f'{userID}_tweets.csv', index=True)

    def obtainScreenshots(self, userID, sleepTime):
        base_url = f'https:/twitter.com/{userID}/status'
        for idx, tweet in enumerate(self.tweets):
            id_tweet = tweet.id
            url = f'{base_url}/{id_tweet}'
            print(url)
            self.takeScreenshots(url, userID, id_tweet, sleepTime)

    def takeScreenshots(self, url, userID, id_tweet, sleepTime):
        wd.get(url)
        time.sleep(sleepTime)
        wd.save_screenshot(f'/content/savedscreenshot/{userID}-{id_tweet}.png')

    def create_dir(self):
        if not os.path.exists('savedscreenshot'):
            print(f'Creating project savedscreenshot')
            os.makedirs('savedscreenshot')

    def run(self):
        try:
            self.initTweet(self.userID)
            self.allTweets(self.userID)
            self.write_file(self.tweets)
            if (self.verbose):
                self.create_dir()
                self.obtainScreenshots(self.userID, self.sleepTime)
        except tw.TweepError:
            print("Protected Account")
            pass


In [ ]:
if __name__ == '__main__':
    """
    userIDs = ['drmuss', 'DrIqbalSarwar','khairul_hafidz', 'rafidah72 ','mafeitz','MYJantung','DrMahyuddin','drafidahyusof','suhazeli','khalilpsych','ctsarahaishah',
               'afidasohana','muhammadeizat','zainaladwin','kay_alia','Lael_Ophthal','drihsansuhaidi','DrAnwarFazal','fdcaraku3800','syariza81','aimirahayu','affick_rahim',
               'Zafrul_Zamberi','DrRadzi Surgery','wanie_dagreat','fiezanizmohamed','hafsah_sazali','FirdausIzzani','RahmatPsych','khiddir89','ib_shah','qonberg',
               'azamspark','kay_haliq','AmiraRaup','wahidatul_abdah','marmellow','izzahrph','jazlinasyahrul','SmSmaudio','MarAzlan']
    """
    """
MKNJPM, MyHEALTHKKM, kkmm_gov, jpmgov_, KDNPUTRAJAYA,
JKJAVMY, DrAdhamBaba, MuhyiddinYassin, DGHisham, IsmailSabri60
RTM_Malaysia, AnnuarMusa, 501Awani, bharianmy,
hannahyeoh, rafidah72, DrAnwarFazal, MedTweetMYHQ, khairul_hafidz, BuletinTV3
ChristinaYong6, loyingru, SarehParangiMD, radzi_dr, ClinicalRsrchMY,
Khairykj, SandboxMalaysia, sputnikvaccine, NIHMalaysia, protecthealthco,
trussliz, SelveeRamasamy, DrMahHS, malaysianmedic1, angepratt,
takeshi_kasai, codebluenews, imokman, DrDzul, HarithIskander,
UMonline, fmtoday, SinarOnline, mkini_bm,
"""
    userIDs = ['DrAdhamBaba', 'MuhyiddinYassin', 'AnnuarMusa', 'ChristinaYong6', 'SandboxMalaysia', 'protecthealthco']
    for userID in userIDs:
        tw_evidence = TwitterInformation(userID, 1, False)
        tw_evidence.run()

In [ ]:
!zip -r /content/Culprits2.zip /content
from google.colab import files


In [ ]:
## jpmgov_
"""
MKNJPM, MyHEALTHKKM, kkmm_gov, jpmgov_, KDNPUTRAJAYA,
JKJAVMY, DrAdhamBaba, MuhyiddinYassin, DGHisham, IsmailSabri60
RTM_Malaysia, AnnuarMusa, 501Awani, bharianmy,
hannahyeoh, rafidah72, DrAnwarFazal, MedTweetMYHQ, khairul_hafidz, BuletinTV3
ChristinaYong6, loyingru, SarehParangiMD, radzi_dr, ClinicalRsrchMY,
Khairykj, SandboxMalaysia, sputnikvaccine, NIHMalaysia, protecthealthco,
trussliz, SelveeRamasamy, DrMahHS, malaysianmedic1, angepratt,
takeshi_kasai, codebluenews, imokman, DrDzul, HarithIskander,
UMonline, fmtoday,
"""
keys = {
    "consumer_key": 'PFntvVTEGYxOj0DpsQGLfcHih',
    "consumer_secret": 'qaMWaTihgx7WKCxs2CWf6VkJrGQMaQyBkd22KW6vhdHpRmO7yg',
    "access_token": '994786472900333570-6qqX2SH8XbHXAbJRkWdwzge4A7xktPo',
    "access_token_secret": 'ZryyU4n78wJyUQaNVDPNR7X7a7XKDAOtxy768SCMH2Y2E'
}
# edit these three variables
user = 'malaysianmedic1'
start = datetime.datetime(2020, 1, 1)  # year, month, day
end = datetime.datetime(2021, 11, 25)  # year, month, day

# only edit these if you're having problems
delay = 1  # time to wait on each page load before reading the page
driver = webdriver.Chrome('chromedriver',options=chrome_options)  # options are Chrome() Firefox() Safari()


# don't mess with this stuff
twitter_ids_filename = 'all_ids.json'
days = (end - start).days + 1
id_selector = 'css-4rbku5.css-18t94o4.css-901oao.r-14j79pv.r-1loqt21.r-1q142lx.r-37j5jr.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-3s2u2q.r-qvutc0'
tweet_selector = 'css-901oao.css-16my406.r-poiln3.r-bcqeeo.r-qvutc0'
user = user.lower()
ids = []

def format_day(date):
    day = '0' + str(date.day) if len(str(date.day)) == 1 else str(date.day)
    month = '0' + str(date.month) if len(str(date.month)) == 1 else str(date.month)
    year = str(date.year)
    return '-'.join([year, month, day])

def form_url(since, until):
    p1 = 'https://twitter.com/search?f=tweets&vertical=default&q=(from%3A'
    p2 =  user + ')%20since%3A' + since + '%20until%3A' + until + 'include%3Aretweets&src=typd'
    return p1 + p2

def increment_day(date, i):
    return date + datetime.timedelta(days=i)

for day in range(days):
    d1 = format_day(increment_day(start, 0))
    d2 = format_day(increment_day(start, 1))
    url = form_url(d1, d2)
    print(url)
    print(d1)
    driver.get(url)
    sleep(delay)

    try:

        #found_tweets = driver.find_elements_by_css_selector(tweet_selector)
        SCROLL_PAUSE_TIME = 1

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME/4)
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)
            counter = 0
            try:
            #id = tweet.find_element_by_class_name(id_selector).get_attribute('href').split('/')[-1]
            #idlst = driver.find_elements(By.CLASS_NAME, id_selector)
                idlst = driver.find_elements(By.TAG_NAME,'a')
                time.sleep(SCROLL_PAUSE_TIME/4)
                for id in idlst:
                    link = ''
                    try:
                        link = (id.get_attribute('href'))

                        if 'status' in link and not 'photo' in link:
                            ids.append(link.split('/')[-1])
                            counter = counter + 1
                    except:
                        print('NA')
                print('{} found'.format(counter))
            except StaleElementReferenceException as e:
                print('lost element reference', tweet)


            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        time.sleep(2)
        #print(driver.page_source)
        #found_tweets = driver.find_elements(By.CLASS_NAME, tweet_selector)

        """
        increment = 10
        #print(found_tweets)
        while len(found_tweets) >= increment:
            print('scrolling down to load more tweets')
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(delay)
            found_tweets.append(driver.find_elements(By.CLASS_NAME, tweet_selector))
            increment += 10
        """


        #print('{} tweets found'.format(len(found_tweets)))

    except NoSuchElementException:
        print('no tweets on this day')

    start = increment_day(start, 1)

"""
try:
    with open(twitter_ids_filename) as f:
        all_ids = ids + json.load(f)
        data_to_write = list(set(all_ids))
        print('tweets found on this scrape: ', len(ids))
        print('total tweet count: ', len(data_to_write))
except FileNotFoundError:
    with open(twitter_ids_filename, 'w') as f:
        all_ids = ids
        data_to_write = list(set(all_ids))
        print('tweets found on this scrape: ', len(ids))
        print('total tweet count: ', len(data_to_write))

with open(twitter_ids_filename, 'w') as outfile:
    json.dump(data_to_write, outfile)
"""
with open(twitter_ids_filename, 'w') as outfile:
    json.dump(list(set(ids)), outfile)
print('all done here')
driver.close()

https://twitter.com/search?f=tweets&vertical=default&q=(from%3Amalaysianmedic1)%20since%3A2020-01-01%20until%3A2020-01-02include%3Aretweets&src=typd
2020-01-01
2 found
2 found
https://twitter.com/search?f=tweets&vertical=default&q=(from%3Amalaysianmedic1)%20since%3A2020-01-02%20until%3A2020-01-03include%3Aretweets&src=typd
2020-01-02
0 found
https://twitter.com/search?f=tweets&vertical=default&q=(from%3Amalaysianmedic1)%20since%3A2020-01-03%20until%3A2020-01-04include%3Aretweets&src=typd
2020-01-03
1 found
https://twitter.com/search?f=tweets&vertical=default&q=(from%3Amalaysianmedic1)%20since%3A2020-01-04%20until%3A2020-01-05include%3Aretweets&src=typd
2020-01-04
0 found
https://twitter.com/search?f=tweets&vertical=default&q=(from%3Amalaysianmedic1)%20since%3A2020-01-05%20until%3A2020-01-06include%3Aretweets&src=typd
2020-01-05
0 found
https://twitter.com/search?f=tweets&vertical=default&q=(from%3Amalaysianmedic1)%20since%3A2020-01-06%20until%3A2020-01-07include%3Aretweets&src=typd
202

In [ ]:
auth = tweepy.OAuthHandler(keys['consumer_key'], keys['consumer_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth)
user = user.lower()
output_file = '{}.json'.format(user)
output_file_short = '{}_short.json'.format(user)
compression = zipfile.ZIP_DEFLATED

with open('all_ids.json') as f:
    ids = json.load(f)

print('total ids: {}'.format(len(ids)))

all_data = []
start = 0
end = 100
limit = len(ids)
i = math.ceil(limit / 100)

for go in range(i):
    print('currently getting {} - {}'.format(start, end))
    sleep(6)  # needed to prevent hitting API rate limit
    id_batch = ids[start:end]
    start += 100
    end += 100
    tweets = api.statuses_lookup(id_batch)
    for tweet in tweets:
        all_data.append(dict(tweet._json))
print(all_data)
print('metadata collection complete')
print('creating master json file')
with open(output_file, 'w') as outfile:
    json.dump(all_data, outfile)

print('creating ziped master json file')
zf = zipfile.ZipFile('{}.zip'.format(user), mode='w')
zf.write(output_file, compress_type=compression)
zf.close()

results = []

def is_retweet(entry):
    return 'retweeted_status' in entry.keys()

def get_source(entry):
    s = f'https://twitter.com/{user}/status/{entry}'
    return s

with open(output_file) as json_data:
    data = json.load(json_data)
    for entry in data:
        t = {
            "created_at": entry["created_at"],
            "text": entry["text"],
            "in_reply_to_screen_name": entry["in_reply_to_screen_name"],
            "retweet_count": entry["retweet_count"],
            "favorite_count": entry["favorite_count"],
            "source": get_source(entry["id_str"]),
            "id_str": entry["id_str"],
            "is_retweet": is_retweet(entry)
        }
        results.append(t)

print('creating minimized json master file')
with open(output_file_short, 'w') as outfile:
    json.dump(results, outfile)

with open(output_file_short) as master_file:
    data = json.load(master_file)
    #fields = ["favorite_count", "source", "text", "in_reply_to_screen_name", "is_retweet", "created_at", "retweet_count", "id_str"]
    fields = ["favorite_count", "source", "text", "created_at", "retweet_count", "id_str"]
    print('creating CSV version of minimized json master file')
    with open('{}.csv'.format(user), 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields)
        writer.writeheader()
        for x in results:
            #f.writerow([x["favorite_count"], x["source"], x["text"], x["in_reply_to_screen_name"], x["is_retweet"], x["created_at"], x["retweet_count"], x["id_str"]])

            small_dict = { your_key: x[your_key] for your_key in fields }
            writer.writerow(small_dict)